# Interdiction de plus courts chemins

MPRO 2018/2018 - RORT

*Guillaume CROGNIER, Guillaume DALLE, Olivier RIGAL*

## 1. Partie programmation linéaire

Installer les packages nécessaires

In [1]:
using Pkg
# Pkg.update()
# Pkg.add("JuMP")
# Pkg.add("Cbc")

Importer les packages

In [2]:
using JuMP
using Cbc

Importer le générateur d'instances

In [3]:
include("instances_io.jl");

### 1.1. Premier programme

Génération de l'instance

In [4]:
l = 5 # number of lines in the grid-like graph
c = 5 # number of columns in the grid-like graph
k = 10 # number of edges the adversary can penalize
maxc = 5 # range of initial edge cost
maxd = 50 # range of penalized edge cost
sv = generate(l, c, k, maxc, maxd);

**Création du modèle**

Liste des arcs

In [5]:
arcs = [(u, v) for u in 1:sv.n for v in 1:sv.n if sv.adj[u][v]];

Ajout de l'objectif et des variables

In [6]:
m = Model(solver=CbcSolver())

x = @variable(m, x[arcs], Bin)
@variable(m, pi[1:sv.n])

s, t = 1, sv.n # source and target nodes
@objective(m, Max, pi[t] - pi[s])

print(m)

Max pi[27] - pi[1]
Subject to
 x[i] ∈ {0,1} ∀ i ∈ {(1, 2),(1, 3),…,(25, 27),(26, 27)}
 pi[i] ∀ i ∈ {1,2,…,26,27}


**Ajout des contraintes**

L'adversaire ne peut pas pénaliser plus de $k$ arcs

In [7]:
@constraint(m, sum(x) <= k);

La pénalisation augmente le coût $c(u, v)$ d'un arc de $d(u, v)$

In [8]:
for (u, v) in arcs
    @constraint(m, pi[v] - pi[u] - sv.d[u][v] * x[(u, v)] <= sv.c[u][v])
end

**Résolution**

In [9]:
status = solve(m)
obj = getobjectivevalue(m)
println("Status ", status)
println("Objective ", obj)

Status Optimal
Objective 38.000000000001
